In [13]:
import pandas as pd

In [ ]:
data = pd.read_excel("data/testSet-1000.xlsx")

In [17]:
data.columns

Index(['id', 'title given by manchine', 'Y/N', 'original title'], dtype='object')

In [6]:
from transformers import BertTokenizer, BertForSequenceClassification

# 加载分词器 - 从原始预训练模型仓库加载
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/home/kapibala/miniconda3/envs/d2l-env/lib/python3.9/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
model_dir = 'results/checkpoint-7265'
model = BertForSequenceClassification.from_pretrained(model_dir)

In [9]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [10]:
def predict(texts):
    # texts是一个字符串列表，如 ["I love this product!", "This was terrible."]
    enc = tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=64,
        return_tensors='pt'
    ).to(device)

    with torch.no_grad():
        outputs = model(**enc)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1).cpu().numpy()
    return preds

In [12]:
test_texts = [
    "This product is wonderful, I really like it.",
    "Heuristic Colour Assignment Strategies for Merge Models in Graph Colouring"
]

predictions = predict(test_texts)
for t, p in zip(test_texts, predictions):
    label_str = "Positive" if p == 1 else "Negative"
    print(f"Text: {t}\nPrediction: {label_str}\n")


Text: This product is wonderful, I really like it.
Prediction: Negative

Text: Heuristic Colour Assignment Strategies for Merge Models in Graph Colouring
Prediction: Positive

